In [1]:
#NB!https://github.com/estnltk/estnltk/blob/master/README.md 
# conda activate py37 esmalt!
import urllib.request, os
from bs4 import BeautifulSoup
import pandas as pd
from pathlib import Path
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import shutil
import os

In [2]:
data = pd.read_csv("Pohitabel.csv")
data.head()

,Kohtuasja_ID,Kohtuasja_nr,Lõigu sisu,Pealkiri1,Tag
0,0,C‑416/20 PPU,EUROOPA KOHTU OTSUS (neljas koda),Koda,C19Centre
1,0,C‑416/20 PPU,17. detsember 2020,Kuupaev,C19Centre
2,0,C‑416/20 PPU,Eelotsusetaotlus – Eelotsuse kiirmenetlus – Po...,Viited,C71Indicateur
3,0,C‑416/20 PPU,TR,Poolte_nimed,C02AlineaAltA
4,0,C‑416/20 PPU,"Generalstaatsanwaltschaft Hamburg,",Poolte_nimed,C02AlineaAltA


In [4]:
column_list=data['Lõigu sisu'].to_list()
column_list[0:100]

['EUROOPA KOHTU OTSUS (neljas koda)',
 '17. detsember 2020',
 'Eelotsusetaotlus – Eelotsuse kiirmenetlus – Politseikoostöö ja õigusalane koostöö kriminaalasjades – Raamotsus 2002/584/JSK – Euroopa vahistamismäärus – Artikli 4a lõige 1 – Liikmesriikidevaheline üleandmiskord – Täitmise tingimused – Täitmata jätmise vabatahtlikud alused – Erandid – Täitmise kohustuslikkus – Tagaselja mõistetud karistus – Kahtlustatava või süüdistatava põgenemine – Direktiiv (EL) 2016/343 – Artiklid 8 ja 9 – Õigus viibida kohtulikul arutelul – Nõuded tagaselja süüdimõistmise puhul – Kontrollimine süüdimõistetu üleandmisel',
 'TR',
 'Generalstaatsanwaltschaft Hamburg,',
 'saksa',
 '–        Generalstaatsanwaltschaft Hamburg, esindaja: J. Fröhlich,',
 '–        Saksamaa valitsus, esindajad: J. Möller, M. Hellmann ja F. Halabi,',
 '–        Rumeenia valitsus, esindajad: E. Gane, L.‑E. Batagoi ja A. Wellman,',
 '–        Poola valitsus, esindajad: B. Majczyna ja J. Sawicka,',
 '–        Euroopa Komisjon, esind

In [5]:
column_list_low=[str(x).lower() for x in column_list]   
column_list_low[0:100]

['euroopa kohtu otsus (neljas koda)',
 '17. detsember 2020',
 'eelotsusetaotlus – eelotsuse kiirmenetlus – politseikoostöö ja õigusalane koostöö kriminaalasjades – raamotsus 2002/584/jsk – euroopa vahistamismäärus – artikli 4a lõige 1 – liikmesriikidevaheline üleandmiskord – täitmise tingimused – täitmata jätmise vabatahtlikud alused – erandid – täitmise kohustuslikkus – tagaselja mõistetud karistus – kahtlustatava või süüdistatava põgenemine – direktiiv (el) 2016/343 – artiklid 8 ja 9 – õigus viibida kohtulikul arutelul – nõuded tagaselja süüdimõistmise puhul – kontrollimine süüdimõistetu üleandmisel',
 'tr',
 'generalstaatsanwaltschaft hamburg,',
 'saksa',
 '–        generalstaatsanwaltschaft hamburg, esindaja: j. fröhlich,',
 '–        saksamaa valitsus, esindajad: j. möller, m. hellmann ja f. halabi,',
 '–        rumeenia valitsus, esindajad: e. gane, l.‑e. batagoi ja a. wellman,',
 '–        poola valitsus, esindajad: b. majczyna ja j. sawicka,',
 '–        euroopa komisjon, esind

In [17]:
column_list_low[11]

'1        eelotsusetaotlus käsitleb euroopa parlamendi ja nõukogu 9. märtsi 2016. aasta direktiivi (el) 2016/343, millega tugevdatakse süütuse presumptsiooni teatavaid aspekte ja õigust viibida kriminaalmenetluses kohtulikul arutelul (elt 2016, l 65, lk 1), artiklite 8 ja 9 tõlgendamist.'

In [6]:
# https://colab.research.google.com/drive/1Uh-w_oCllFMaUQOBZVL6xsdSyOfXBTxK?usp=sharing#scrollTo=AUORCu_ztPwG 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vectorizer = CountVectorizer()

In [7]:
#Meetod teksti sõnestamiseks ja lemmatiseerimiseks estnltk abil
from estnltk import Text
def lemmatize_with_estnltk(input_string):
    lemmas = []
    text = Text(input_string).tag_layer()
    for word in text.morph_analysis:
        lemmas.append(word.lemma[0])
    return lemmas

vectorizer = TfidfVectorizer(tokenizer = lemmatize_with_estnltk)

In [8]:
# Andmete viimine vektorkujule
matrix = vectorizer.fit_transform(column_list_low[0:100])
matrix.shape

(100, 850)

In [9]:
from sklearn.metrics.pairwise import cosine_distances

# Leiame kõigi tekstide kaugused üksteisest
cos_dist = cosine_distances(matrix)
cos_dist

array([[0.        , 1.        , 0.97304778, ..., 0.91350038, 0.96603674,
        0.98734469],
       [1.        , 0.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.97304778, 1.        , 0.        , ..., 0.75142092, 0.84566008,
        0.90535872],
       ...,
       [0.91350038, 1.        , 0.75142092, ..., 0.        , 0.74671191,
        0.67842459],
       [0.96603674, 1.        , 0.84566008, ..., 0.74671191, 0.        ,
        0.6107096 ],
       [0.98734469, 1.        , 0.90535872, ..., 0.67842459, 0.6107096 ,
        0.        ]])

In [18]:
# Uurime tekste, mis on omavahel mõnevõrra sarnased
texts=column_list_low[0:100]
for idx, k in enumerate(cos_dist):
    for idx2, j in enumerate(k):
        if idx2 > idx:
            if j > 0.8 and j < 1:
                print(idx, idx2)
                print(j, '\n\n', texts[idx], '\n\n', texts[idx2], '\n', '-'*40, '\n')

0 2
0.9730477845604999 

 euroopa kohtu otsus (neljas koda) 

 eelotsusetaotlus – eelotsuse kiirmenetlus – politseikoostöö ja õigusalane koostöö kriminaalasjades – raamotsus 2002/584/jsk – euroopa vahistamismäärus – artikli 4a lõige 1 – liikmesriikidevaheline üleandmiskord – täitmise tingimused – täitmata jätmise vabatahtlikud alused – erandid – täitmise kohustuslikkus – tagaselja mõistetud karistus – kahtlustatava või süüdistatava põgenemine – direktiiv (el) 2016/343 – artiklid 8 ja 9 – õigus viibida kohtulikul arutelul – nõuded tagaselja süüdimõistmise puhul – kontrollimine süüdimõistetu üleandmisel 
 ---------------------------------------- 

0 10
0.9693466901051063 

 euroopa kohtu otsus (neljas koda) 

 –        euroopa komisjon, esindajad: c. ladenburger, m. wasmeier ja s. grünheid, 
 ---------------------------------------- 

0 11
0.8990558314230368 

 euroopa kohtu otsus (neljas koda) 

 1        eelotsusetaotlus käsitleb euroopa parlamendi ja nõukogu 9. märtsi 2016. aasta dire

 ---------------------------------------- 

2 74
0.8608387741767071 

 eelotsusetaotlus – eelotsuse kiirmenetlus – politseikoostöö ja õigusalane koostöö kriminaalasjades – raamotsus 2002/584/jsk – euroopa vahistamismäärus – artikli 4a lõige 1 – liikmesriikidevaheline üleandmiskord – täitmise tingimused – täitmata jätmise vabatahtlikud alused – erandid – täitmise kohustuslikkus – tagaselja mõistetud karistus – kahtlustatava või süüdistatava põgenemine – direktiiv (el) 2016/343 – artiklid 8 ja 9 – õigus viibida kohtulikul arutelul – nõuded tagaselja süüdimõistmise puhul – kontrollimine süüdimõistetu üleandmisel 

 31      sellega seoses tuleb märkida, et nagu nähtub raamotsuse 2002/584 artikli 1 lõigetest 1 ja 2 ning põhjendustest 5 ja 7, on raamotsuse eesmärk asendada liikmesriikidevaheline mitmepoolne väljaandmissüsteem, mis põhineb 13. detsembril 1957 pariisis allkirjastatud väljaandmise euroopa konventsioonil, vastastikuse tunnustamise põhimõttel rajaneva õigusasutustevahelise üleand

6 71
0.9403350082536728 

 –        generalstaatsanwaltschaft hamburg, esindaja: j. fröhlich, 

 28      käesoleval juhul nähtub eelotsusetaotlusest, et eelotsusetaotluse esitanud kohus peab tegema otsuse selle kohta, kas tri üleandmine rumeenia ametiasutustele on õiguspärane seaduse rahvusvahelise õigusabi kohta kriminaalasjades § 83 sätete alusel, millega rakendatakse raamotsuse 2002/584 artiklit 4a saksa õiguses. 
 ---------------------------------------- 

6 72
0.8974833348454981 

 –        generalstaatsanwaltschaft hamburg, esindaja: j. fröhlich, 

 29      eelotsusetaotluse esitanud kohtu hinnangul on üleandmiseks vajalikud tingimused täidetud, kuna esiteks on see isik teadlikult hoidunud menetlustest – mille tulemusel on tehtud need euroopa vahistamismäärused, mida see kohus peab täitma –, põgenedes saksamaale ja takistades seega talle kohtukutse isiklikult kättetoimetamist, ning teiseks esindas seda isikut esimeses kohtuastmes tema valitud advokaat ja apellatsiooniastmes kohtu

0.8694432326068522 

 –        rumeenia valitsus, esindajad: e. gane, l.‑e. batagoi ja a. wellman, 

 37      nimetatud artikkel 4a ühtlustab tagaseljaotsuste kohta tehtud euroopa vahistamismääruse täitmise tingimusi, kusjuures vahistamismäärus peegeldab kõikide liikmesriikide saavutatud konsensust selles osas, milline ulatus anda liidu õiguse alusel neile menetluslikele õigustele, mis on tagaselja süüdimõistetul, kelle suhtes on tehtud euroopa vahistamismäärus (26. veebruari 2013. aasta kohtuotsus melloni, c‑399/11, eu:c:2013:107, punkt 62). 
 ---------------------------------------- 

8 81
0.8230393240783043 

 –        rumeenia valitsus, esindajad: e. gane, l.‑e. batagoi ja a. wellman, 

 38      nii tuleneb juba raamotsuse 2002/584 artikli 4a lõike 1 sõnastusest, et vahistamismäärust täitev õigusasutus võib keelduda täitmast euroopa vahistamismäärust, mis on tehtud vabadusekaotuse või vabadust piirava julgeolekumeetme täitmiseks, kui isik ei ilmunud isiklikult kohtulikule arutelule

0.8825891552390625 

 1        eelotsusetaotlus käsitleb euroopa parlamendi ja nõukogu 9. märtsi 2016. aasta direktiivi (el) 2016/343, millega tugevdatakse süütuse presumptsiooni teatavaid aspekte ja õigust viibida kriminaalmenetluses kohtulikul arutelul (elt 2016, l 65, lk 1), artiklite 8 ja 9 tõlgendamist. 

 2        taotlus on esitatud selliste euroopa vahistamismääruste täitmise menetluses saksamaal, mille on teinud judecătoria deva (deva esimese astme kohus, rumeenia) 7. oktoobril 2019 ja tribunalul hunedoara (hunedoara esimese astme kohus, rumeenia) 4. veebruaril 2020 vabadusekaotuslike karistuste täideviimiseks, mille on tr‑le tagaselja mõistnud rumeenia kohtud. 
 ---------------------------------------- 

11 14
0.9452765726217054 

 1        eelotsusetaotlus käsitleb euroopa parlamendi ja nõukogu 9. märtsi 2016. aasta direktiivi (el) 2016/343, millega tugevdatakse süütuse presumptsiooni teatavaid aspekte ja õigust viibida kriminaalmenetluses kohtulikul arutelul (elt 2016, l 65

...

0.8278020339789589 

 11      eelotsusetaotlusest nähtub, et eelotsusetaotluse esitanud kohtu hanseatisches oberlandesgericht hamburgi (hamburgi liidumaa kõrgeim üldkohus, saksamaa) poole on pöördunud rumeenia ametiasutused kahe euroopa vahistamismäärusega, mis tehti vastavalt 7. oktoobril 2019 ja 4. veebruaril 2020 rumeenia kodaniku tri üleandmiseks, et viia täide vabadusekaotuslikud karistused, mille rumeenia kohtud on talle tagaselja mõistnud. tr viibib alates 31. märtsist 2020 hamburgis (saksamaa) väljaandmisvahistuses. 

 38      nii tuleneb juba raamotsuse 2002/584 artikli 4a lõike 1 sõnastusest, et vahistamismäärust täitev õigusasutus võib keelduda täitmast euroopa vahistamismäärust, mis on tehtud vabadusekaotuse või vabadust piirava julgeolekumeetme täitmiseks, kui isik ei ilmunud isiklikult kohtulikule arutelule, mille tulemusel otsus tehti, välja arvatud juhul, kui euroopa vahistamismääruses märgitakse, et selle sätte punktides a–d sätestatud tingimused on täidetud (10. augus

 ---------------------------------------- 

53 80
0.8864706998315401 

 –        teine lõplik kohtuotsus, millega mõisteti tagaselja nelja-aastane vangistus, millest tal tuleb veel ära kanda kaks aastat ja neli kuud ning mida on pikendatud 1786päevase veel kandmata karistuse võrra kuritegelikku ühendusse kuulumise eest, narkootiliste ainete käitlemise eest, mis on toime pandud grupi poolt, kahe liiklussüüteo eest ja tervisekahjustuse tekitamise eest. 

 37      nimetatud artikkel 4a ühtlustab tagaseljaotsuste kohta tehtud euroopa vahistamismääruse täitmise tingimusi, kusjuures vahistamismäärus peegeldab kõikide liikmesriikide saavutatud konsensust selles osas, milline ulatus anda liidu õiguse alusel neile menetluslikele õigustele, mis on tagaselja süüdimõistetul, kelle suhtes on tehtud euroopa vahistamismäärus (26. veebruari 2013. aasta kohtuotsus melloni, c‑399/11, eu:c:2013:107, punkt 62). 
 ---------------------------------------- 

53 81
0.8733190319954204 

 –        teine lõplik 

 ---------------------------------------- 

58 64
0.9630220722305884 

 17      rumeenia ametiasutused keeldusid siiski rahuldamast saksamaa ametiasutuste taotlust saada tagatised kõnealuste kriminaalmenetluste uuendamiseks, kuna trile olid kohtukutsed nõuetekohaselt kätte toimetatud ja süüdimõistvaid kohtuotsuseid ei saa seega rumeenia kriminaalmenetluse seadustiku alusel uuesti läbi vaadata. 

 22      eelotsusetaotluse esitanud kohus palub euroopa kohtul lahendada eelotsusetaotlus euroopa kohtu kodukorra artikli 107 alusel kiirmenetluses. 
 ---------------------------------------- 

58 65
0.9051073421362763 

 17      rumeenia ametiasutused keeldusid siiski rahuldamast saksamaa ametiasutuste taotlust saada tagatised kõnealuste kriminaalmenetluste uuendamiseks, kuna trile olid kohtukutsed nõuetekohaselt kätte toimetatud ja süüdimõistvaid kohtuotsuseid ei saa seega rumeenia kriminaalmenetluse seadustiku alusel uuesti läbi vaadata. 

 23      sellega seoses tuleb esimesena märkida, et 

62 83
0.9204137229293012 

 21      neil asjaoludel otsustas hanseatisches oberlandesgericht hamburg (hamburgi liidumaa kõrgeim üldkohus) menetluse peatada ja esitada euroopa kohtule järgmise eelotsuse küsimuse: 

 40      nagu euroopa kohus on juba märkinud, kasutas liidu seadusandja lahendust, mis seisneb selles, et nähakse ette ammendav loetelu olukordadest, mil tagaseljaotsuse täitmisele pööramiseks tehtud euroopa vahistamismäärust tuleb käsitada nii, et see ei riku kaitseõigusi (26. veebruari 2013. aasta kohtuotsus melloni, c‑399/11, eu:c:2013:107, punkt 44). 
 ---------------------------------------- 

62 84
0.9436166517838954 

 21      neil asjaoludel otsustas hanseatisches oberlandesgericht hamburg (hamburgi liidumaa kõrgeim üldkohus) menetluse peatada ja esitada euroopa kohtule järgmise eelotsuse küsimuse: 

 41      sellest järeldub, et vahistamismäärust täitev õigusasutus on juhul, kui on kindlaks tehtud, et üks raamotsuse 2002/584 artikli 4a lõike 1 punktides a, b, c või d

71 80
0.8141185568560829 

 28      käesoleval juhul nähtub eelotsusetaotlusest, et eelotsusetaotluse esitanud kohus peab tegema otsuse selle kohta, kas tri üleandmine rumeenia ametiasutustele on õiguspärane seaduse rahvusvahelise õigusabi kohta kriminaalasjades § 83 sätete alusel, millega rakendatakse raamotsuse 2002/584 artiklit 4a saksa õiguses. 

 37      nimetatud artikkel 4a ühtlustab tagaseljaotsuste kohta tehtud euroopa vahistamismääruse täitmise tingimusi, kusjuures vahistamismäärus peegeldab kõikide liikmesriikide saavutatud konsensust selles osas, milline ulatus anda liidu õiguse alusel neile menetluslikele õigustele, mis on tagaselja süüdimõistetul, kelle suhtes on tehtud euroopa vahistamismäärus (26. veebruari 2013. aasta kohtuotsus melloni, c‑399/11, eu:c:2013:107, punkt 62). 
 ---------------------------------------- 

71 82
0.8662492399304778 

 28      käesoleval juhul nähtub eelotsusetaotlusest, et eelotsusetaotluse esitanud kohus peab tegema otsuse selle kohta, kas t

In [ ]:
# vaja oleks siia veel võrdlusi teha!